In [102]:
import requests
from bs4 import BeautifulSoup
import re
import time

# use kpop juice rankings. They have many problems like replacing Keerlejun with Cole, and having non-unique identifiers for the two Neils and Jeremys. 
base_url = 'https://kpopjuice.com/en/youthwithyou3/ranking?type=official&ranking_id='

# iterate over weeks
for j in range(1,4):
    url = base_url + str(j)
    
    # get text
    print(url)
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    
    trainees = []
    
    # rank trainees by the order in which they appear in the list
    i = 1
    for t in soup.find_all('p', {"class": "uk-text-small"}):
        name = re.sub(r'\([^)]*\)', '', t.text.replace('\n', ''))
        trainees.append((name, int(i)))
        i += 1
        
    data_out = pd.DataFrame(trainees[:-1], columns = ['trainee', 'ranking_week' + str(2*j)])
    
    # fix local error
    if j == 3:
        data_out.trainee.iloc[35] = 'Keerlijun'
    
    # set first data
    if j == 1:
        t_data = data_out
    else:
        # merge based on trainee name
        t_data = t_data.merge(data_out, on = 'trainee', how='left')

    print(t_data)
    
    # sleep to avoid overloading the server
    time.sleep(1)
    


https://kpopjuice.com/en/youthwithyou3/ranking?type=official&ranking_id=1
          trainee  ranking_week2
0          Rimiko              1
1           Tony               2
2            Jojo              3
3    Lian Huaiwei              4
4      Sun Yihang              5
..            ...            ...
113        Bernie            114
114         Johan            115
115     Cao Yuxue            116
116     The Dream            117
117          Cole            118

[118 rows x 2 columns]
https://kpopjuice.com/en/youthwithyou3/ranking?type=official&ranking_id=2
          trainee  ranking_week2  ranking_week4
0          Rimiko              1              2
1           Tony               2              1
2            Jojo              3              5
3    Lian Huaiwei              4              6
4      Sun Yihang              5              7
..            ...            ...            ...
117        Bernie            114            104
118         Johan            115             92


/Users/johnnyma/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


          trainee  ranking_week2  ranking_week4  ranking_week6
0          Rimiko              1              2              2
1           Tony               2              1              1
2            Jojo              3              5              5
3    Lian Huaiwei              4              6              6
4      Sun Yihang              5              7              7
..            ...            ...            ...            ...
125        Bernie            114            104            111
126         Johan            115             92             98
127     Cao Yuxue            116            102            115
128     The Dream            117             90             91
129          Cole            118             48             58

[130 rows x 4 columns]


In [103]:
# output to csv
t_data.to_csv('trainee_data.csv')